In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, TimeDistributed, Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
#create a training dataset for the model


def rpn_layer(base_layers, num_anchors):
    """Create a convolution layer to predict object scores and bounding boxes."""
    x = Conv2D(512, (3, 3), padding='same', activation='relu', kernel_initializer='normal', name='rpn_conv1')(base_layers)
    x_class = Conv2D(num_anchors, (1, 1), activation='sigmoid', kernel_initializer='uniform', name='rpn_out_class')(x)
    x_regr = Conv2D(num_anchors * 4, (1, 1), activation='linear', kernel_initializer='zero', name='rpn_out_regress')(x)
    return [x_class, x_regr, base_layers]

def classifier_layer(base_layers, input_rois, num_rois, num_classes):
    """Define the classifier layer to classify object types and bounding boxes"""
    pooling_regions = 7
    x = TimeDistributed(tf.keras.layers.MaxPooling2D(pool_size=(7, 7)), input_shape=(num_rois,7,7,512))(input_rois)

    x = TimeDistributed(Flatten())(x)
    x = TimeDistributed(Dense(4096, activation='relu'))(x)
    x = TimeDistributed(Dropout(0.5))(x)
    x = TimeDistributed(Dense(4096, activation='relu'))(x)
    x = TimeDistributed(Dropout(0.5))(x)
    out_class = TimeDistributed(Dense(num_classes, activation='softmax', kernel_initializer='zero'), name='dense_class')(x)
    out_regr = TimeDistributed(Dense(num_classes * 4, activation='linear', kernel_initializer='zero'), name='dense_regress')(x)
    return [out_class, out_regr]

def get_faster_rcnn_model():
    num_anchors = 9
    num_rois = 32
    num_classes = 4  # Adjust based on your dataset (including background)
    pooling_regions = 7
    # Base network (VGG16)
    base_model = VGG16(weights='imagenet', include_top=False)
    base_layers = base_model.get_layer('block5_conv3').output

    # Define inputs
    input_shape = base_model.input_shape[1:]
    img_input = Input(shape=input_shape)
    roi_input = Input(shape=(num_rois, pooling_regions, pooling_regions, 512))

    # Create RPN
    rpn_classes, rpn_regressors, shared_layers = rpn_layer(base_layers, num_anchors)

    # Create classifier
    classifier_classes, classifier_regressors = classifier_layer(shared_layers, roi_input, num_rois, num_classes)

    # Creating models
    model_rpn = Model(inputs=img_input, outputs=[rpn_classes, rpn_regressors])
    model_classifier = Model(inputs=[img_input, roi_input], outputs=[classifier_classes, classifier_regressors])

    # Compile models
    model_rpn.compile(optimizer='adam', loss=['binary_crossentropy', 'mean_squared_error'], metrics=['accuracy'])
    model_classifier.compile(optimizer='adam', loss=['categorical_crossentropy', 'mean_squared_error'], metrics=['accuracy'])

    return model_rpn, model_classifier



def fit_and_evaluate(t_x, val_x, t_y, val_y, t_box, val_box, EPOCHS, BATCH_SIZE, class_w):
    # Assuming `get_faster_rcnn_model()` is a function that builds and compiles the Faster R-CNN model
    model_rpn, model_classifier = get_faster_rcnn_model()
    
    # Callbacks for training
    erlstp = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1)
    model_checkpoint = ModelCheckpoint('faster_rcnn_model.h5', monitor='val_loss', save_best_only=True, verbose=1)

    # Training the RPN
    print("Training RPN...")
    model_rpn.fit(t_x, [t_y, t_box], batch_size=BATCH_SIZE, epochs=EPOCHS,
                  validation_data=(val_x, [val_y, val_box]), 
                  class_weight=class_w, callbacks=[erlstp, reduce_lr, model_checkpoint], verbose=1)

    # Training the Classifier
    print("Training Classifier...")
    model_classifier.fit(t_x, [t_y, t_box], batch_size=BATCH_SIZE, epochs=EPOCHS,
                         validation_data=(val_x, [val_y, val_box]), 
                         class_weight=class_w, callbacks=[erlstp, reduce_lr, model_checkpoint], verbose=1)

    # Evaluate the model
    print("\nValidation Score: ", model_classifier.evaluate(val_x, [val_y, val_box]))
    return model_rpn.history, model_classifier.history

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

n_folds = 5
epochs = 100
batch_size = 32
weight = {'class_weight': 1}  # Define as needed based on your class imbalances

model_history = []

for i in range(n_folds):
    print("Training on Fold: ", i + 1)

    # Assuming your data and labels are split into features (X), labels for classification (Y), and bounding boxes (B)
    X_train, X_val, Y_train, Y_val, B_train, B_val = train_test_split(
        X, Y, B, test_size=0.2, random_state=i)  # Ensure you have a proper split for X, Y, and B

    X_train, Y_train, B_train = shuffle(X_train, Y_train, B_train, random_state=i)

    history_rpn, history_classifier = fit_and_evaluate(X_train, X_val, Y_train, Y_val, B_train, B_val, epochs, batch_size, weight)
    model_history.append((history_rpn, history_classifier))
    print("=======" * 12, end="\n\n")


In [1]:
import tensorflow as tf
def evaluate_object_detection(ground_truth_boxes, detected_boxes, detection_methods):
  """
  Evaluates object detection performance for multiple methods.

  Args:
      ground_truth_boxes: List of ground truth bounding boxes for each image.
                          Each box is a list of [y_min, x_min, y_max, x_max] coordinates.
      detected_boxes: List of detected bounding boxes for each image from different methods.
                       Each element is a dictionary with keys as detection methods (strings)
                       and values as lists of detections. Each detection is a list of
                       [y_min, x_min, y_max, x_max, confidence_score] coordinates.
      detection_methods: List of detection method names (strings).

  Returns:
      A dictionary containing precision, recall, and F1-score for each detection method.
  """

  # Initialize empty dictionaries to store evaluation metrics
  precisions = {method: [] for method in detection_methods}
  recalls = {method: [] for method in detection_methods}
  f1_scores = {method: [] for method in detection_methods}

  # Loop through each image
  for gt_boxes, det_boxes_dict in zip(ground_truth_boxes, detected_boxes):
    # Loop through each detection method
    # for method, detections in det_boxes_dict.items():
    for method in detection_methods:
      detections = det_boxes_dict[method]
      # Calculate true positives, false positives, and false negatives
      true_positives = 0
      false_positives = 0
      false_negatives = len(gt_boxes)

      # Loop through each ground truth box
      for gt_box in gt_boxes:
        # Check for overlaps with detected boxes for this method
        iou_max = 0
        for det_box in detections:
          iou = calculate_iou(gt_box, det_box)  # Replace with your IOU calculation function
          iou_max = max(iou_max, iou)

        # Consider a ground truth box detected if IoU is above a threshold (e.g., 0.5)
        if iou_max >= 0.5:
          true_positives += 1
          false_negatives -= 1

      # Calculate false positives from detections without matching ground truth boxes
      false_positives = len(detections) - true_positives

      # Calculate precision, recall, and F1-score
      if true_positives + false_positives > 0:
        precision = true_positives / (true_positives + false_positives)
      else:
        precision = 0.0
      if len(gt_boxes) > 0:
        recall = true_positives / len(gt_boxes)
      else:
        recall = 0.0
      if precision + recall > 0:
        f1_score = 2 * (precision * recall) / (precision + recall)
      else:
        f1_score = 0.0

      # Append metrics for this method and image
      precisions[method].append(precision)
      recalls[method].append(recall)
      f1_scores[method].append(f1_score)

  # Calculate average precision, recall, and F1-score for each method
  average_metrics = {}
  for method in detection_methods:
    average_metrics[method] = {
        "precision": tf.reduce_mean(precisions[method]),
        "recall": tf.reduce_mean(recalls[method]),
        "f1_score": tf.reduce_mean(f1_scores[method]),
    }

  return average_metrics

def calculate_iou(gt_box, det_box):
  """
  Calculates Intersection-over-Union (IoU) between two bounding boxes.

  Args:
      gt_box: Ground truth bounding box (list of [y_min, x_min, y_max, x_max] coordinates).
      det_box: Detected bounding box (list of [y_min, x_min, y_max, x_max, confidence_score] coordinates).

  Returns:
      IoU value (float) between 0 and 1.
  """
  # Calculate area of overlap and area of union
  ymin_intersection = max(gt_box[0], det_box[0])
  xmin_intersection = max(gt_box[1], det_box[1])
  ymax_intersection = min(gt_box[2], det_box[2])
  xmax_intersection = min(gt_box[3], det_box[3])

  area_intersection = max(0, xmax_intersection - xmin_intersection) * max(0, ymax_intersection - ymin_intersection)
  area_gt = (gt_box[2] - gt_box[0]) * (gt_box[3] - gt_box[1])
  area_det = (det_box[2] - det_box[0]) * (det_box[3] - det_box[1])
  area_union = area_gt + area_det - area_intersection

  # Calculate IoU
  iou = area_intersection / (area_union + 1e-10)  # Add a small value to avoid division by zero
  return iou

# Example usage
ground_truth_boxes = [
    [0.2, 0.3, 0.7, 0.8],  # Example ground truth bounding box
    # ... more ground truth boxes
]

detected_boxes = {
    "method_1": [
        [0.1, 0.2, 0.8, 0.9, 0.9],  # Example detection from method 1
        # ... more detections for method 1
    ],
    "method_2": [
        # ... detections from method 2
    ],
    # ... detections from other methods
}

detection_methods = ["method_1", "method_2", ...]  # List of detection method names

evaluation_results = evaluate_object_detection(ground_truth_boxes, detected_boxes, detection_methods)

# Print evaluation results for each method
for method, metrics in evaluation_results.items():
  print(f"Method: {method}")
  print(f"  Precision: {metrics['precision']}")
  print(f"  Recall: {metrics['recall']}")
  print(f"  F1-Score: {metrics['f1_score']}")  



TypeError: string indices must be integers, not 'str'

In [2]:
import numpy as np

def evaluate_detection(gt_boxes, pred_boxes, iou_threshold=0.5):
    """
    Evaluate object detection performance using precision, recall, and F1 score.

    Parameters:
    - gt_boxes: List of ground truth bounding boxes in format [x_min, y_min, x_max, y_max].
    - pred_boxes: List of predicted bounding boxes in format [x_min, y_min, x_max, y_max].
    - iou_threshold: IoU threshold for considering a detection as a true positive.

    Returns:
    - precision: Precision of the detection.
    - recall: Recall of the detection.
    - f1_score: F1 score of the detection.
    """

    if len(gt_boxes) == 0 or len(pred_boxes) == 0:
        return 0, 0, 0

    gt_boxes = np.array(gt_boxes)
    pred_boxes = np.array(pred_boxes)

    true_positives = 0
    false_positives = 0
    false_negatives = 0

    for pred_box in pred_boxes:
        iou = calculate_iou(gt_boxes, pred_box)
        if np.max(iou) >= iou_threshold:
            true_positives += 1
        else:
            false_positives += 1

    false_negatives = len(gt_boxes) - true_positives

    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    
    if precision == 0 or recall == 0:
        f1_score = 0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)

    return precision, recall, f1_score

def calculate_iou(gt_boxes, pred_box):
    """
    Calculate the Intersection over Union (IoU) between a predicted bounding box and a list of ground truth bounding boxes.

    Parameters:
    - gt_boxes: List of ground truth bounding boxes in format [x_min, y_min, x_max, y_max].
    - pred_box: Predicted bounding box in format [x_min, y_min, x_max, y_max].

    Returns:
    - iou: IoU value between the predicted box and each ground truth box.
    """
    x1 = np.maximum(gt_boxes[:, 0], pred_box[0])
    y1 = np.maximum(gt_boxes[:, 1], pred_box[1])
    x2 = np.minimum(gt_boxes[:, 2], pred_box[2])
    y2 = np.minimum(gt_boxes[:, 3], pred_box[3])

    intersection_area = np.maximum(0, x2 - x1) * np.maximum(0, y2 - y1)
    gt_box_area = (gt_boxes[:, 2] - gt_boxes[:, 0]) * (gt_boxes[:, 3] - gt_boxes[:, 1])
    pred_box_area = (pred_box[2] - pred_box[0]) * (pred_box[3] - pred_box[1])

    iou = intersection_area / (gt_box_area + pred_box_area - intersection_area)

    return iou

# Example usage:
gt_boxes = [[100, 100, 200, 200], [300, 300, 400, 400]]
pred_boxes = [[90, 90, 210, 210], [320, 320, 420, 420], [500, 500, 600, 600]]
iou_threshold = 0.5

precision, recall, f1_score = evaluate_detection(gt_boxes, pred_boxes, iou_threshold)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


Precision: 0.3333333333333333
Recall: 0.5
F1 Score: 0.4


In [46]:
import numpy as np
import pandas as pd

def calculation_accuracy(detection_per_frame, ground_truth_per_frame):
   """
   param: detection_per_frame - list of integers representing number of human detections per frame.
   param: ground_truth_per_frame - list of integers representing number of actual humans in the frame.
   """
   if len(detection_per_frame) != len(ground_truth_per_frame):
      raise ValueError("Different number of detections and ground truths")
   
   correct_detections = sum(1 for detected, actual in zip(detection_per_frame, ground_truth_per_frame) if detected == actual)
   accuracy = correct_detections / len(ground_truth_per_frame) 
   return f'{accuracy*100:.2f} %'

def evaluate_detection(gt_boxes, pred_boxes, iou_threshold=0.5):
    """
    Evaluate object detection performance using precision, recall, and F1 score.

    Parameters:
    - gt_boxes: List of ground truth bounding boxes in format [x_min, y_min, x_max, y_max].
    - pred_boxes: List of predicted bounding boxes in format [x_min, y_min, x_max, y_max].
    - iou_threshold: IoU threshold for considering a detection as a true positive.

    Returns:
    - precision: Precision of the detection.
    - recall: Recall of the detection.
    - f1_score: F1 score of the detection.
    """

    if len(gt_boxes) == 0 or len(pred_boxes) == 0:
        return 0, 0, 0

    gt_boxes = np.array(gt_boxes)
    pred_boxes = np.array(pred_boxes)

    true_positives = 0
    false_positives = 0
    false_negatives = 0

    for pred_box in pred_boxes:
        iou = calculate_iou(gt_boxes, pred_box)
        if np.max(iou) >= iou_threshold:
            true_positives += 1
        else:
            false_positives += 1

    false_negatives = len(gt_boxes) - true_positives

    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    
    if precision == 0 or recall == 0:
        f1_score = 0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)

    return precision, recall, f1_score

def calculate_iou(gt_boxes, pred_box):
    """
    Calculate the Intersection over Union (IoU) between a predicted bounding box and a list of ground truth bounding boxes.

    Parameters:
    - gt_boxes: List of ground truth bounding boxes in format [x_min, y_min, x_max, y_max].
    - pred_box: Predicted bounding box in format [x_min, y_min, x_max, y_max].

    Returns:
    - iou: IoU value between the predicted box and each ground truth box.
    """
    x1 = np.maximum(gt_boxes[:, 0], pred_box[0])
    y1 = np.maximum(gt_boxes[:, 1], pred_box[1])
    x2 = np.minimum(gt_boxes[:, 2], pred_box[2])
    y2 = np.minimum(gt_boxes[:, 3], pred_box[3])

    intersection_area = np.maximum(0, x2 - x1) * np.maximum(0, y2 - y1)
    gt_box_area = (gt_boxes[:, 2] - gt_boxes[:, 0]) * (gt_boxes[:, 3] - gt_boxes[:, 1])
    pred_box_area = (pred_box[2] - pred_box[0]) * (pred_box[3] - pred_box[1])
    area_union = gt_box_area + pred_box_area - intersection_area
    iou = intersection_area / area_union 

    return iou

def print_evaluation_table(detection_methods, results, results2):
    """
    Print a table with precision, recall, and F1 scores for multiple detection methods.

    Parameters:
    - detection_methods: List of names of detection methods.
    - results: Dictionary containing evaluation results for each method.
    """
    df = pd.DataFrame(results, index=['Precision', 'Recall', 'F1 Score'])
    df.columns.name = 'Detection Method'
    # Convert accuracy results dictionary to DataFrame
    accuracy_df = pd.DataFrame(results2, index=['Human Detection Accuracy'])
    
    # Merge accuracy DataFrame with main DataFrame
    df = pd.concat([df, accuracy_df])
    # df['Accuracy'] = pd.Series(results2)
    
    print(df)

# Example usage:
gt_boxes = [[100, 100, 200, 200], [300, 300, 400, 400]]
pred_boxes_method1 = [[90, 90, 210, 210], [320, 320, 420, 420], [500, 500, 600, 600]]
pred_boxes_method2 = [[100, 100, 200, 200], [310, 310, 410, 410], [520, 520, 620, 620]]
iou_threshold = 0.5

# Example usage (assuming you have these lists from your video processing)
yolo_detections = [3, 2, 4, 3, 5]  # Example detection counts per frame from YOLO
hog_detections = [2, 2, 4, 3, 5]   # Example detection counts per frame from HOG
ground_truth = [3, 2, 4, 3, 5]     # Actual counts of humans per frame

results = {}
results['YOLOv4'] = evaluate_detection(gt_boxes, pred_boxes_method1, iou_threshold)
results['HOG'] = evaluate_detection(gt_boxes, pred_boxes_method2, iou_threshold)
# print_evaluation_table(['YOLOv4', 'HOG'], results)

results2={}
results2['YOLOv4'] = calculation_accuracy(yolo_detections, ground_truth)
results2['HOG'] = calculation_accuracy(hog_detections, ground_truth)
print_evaluation_table(['YOLOv4', 'HOG'], results, results2)




                            YOLOv4       HOG
Precision                 0.333333  0.666667
Recall                         0.5       1.0
F1 Score                       0.4       0.8
Human Detection Accuracy  100.00 %   80.00 %


In [ ]:
import cv2 
import numpy as np

def detect_objects(video_path, weights_path, config_path, class_names_path): 
    # Initialize the network 
    net = cv2.dnn_DetectionModel(weights_path, config_path) 
    net.setInputSize(416, 416) 
    net.setInputScale(1.0 / 255) 
    net.setInputSwapRB(True)
    # Load class labels
    try:
        with open(class_names_path, "r") as f:
            classes = [line.strip() for line in f.readlines()]
    except FileNotFoundError:
        print("Class names file not found.")
        return

    # Initialize counters and lists for detections
    total_human_detections = 0
    total_non_human_detections = 0
    human_detections_per_frame = []
    non_human_detections_per_frame = []

    # Lists to store predicted bounding boxes
    all_pred_boxes = []

    # Open the video
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video file.")
        return

    while True:
        # Read a frame from the video
        ret, frame = cap.read()
        if not ret:
            break
        
        # Perform detection
        class_ids, confidences, boxes = net.detect(frame, confThreshold=0.5, nmsThreshold=0.4)

        # Store predicted bounding boxes
        all_pred_boxes.append(boxes)
        
        # Initialize counters for detections in the current frame
        frame_human_detections = 0
        frame_non_human_detections = 0
        
        # Count detections and draw bounding boxes
        for (class_id, confidence, box) in zip(class_ids, confidences, boxes):
            class_name = classes[int(class_id)]
            if class_name == 'person':
                # If the detected class is 'person', draw a green bounding box
                color = (0, 255, 0)  # Green color for humans
                frame_human_detections += 1
            else:
                # For non-human objects, use a blue bounding box
                color = (255, 0, 0)  # Blue color for non-human objects
                frame_non_human_detections += 1
            
            # Draw bounding boxes around detected objects
            x, y, w, h = box
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        
        # Update total and per-frame detections
        total_human_detections += frame_human_detections
        total_non_human_detections += frame_non_human_detections
        human_detections_per_frame.append(frame_human_detections)
        non_human_detections_per_frame.append(frame_non_human_detections)
        
        # Optionally, print the number of detections in the current frame
        print(f"Humans detected in this frame: {frame_human_detections}, Non-human objects detected in this frame: {frame_non_human_detections}")
        
        # Show the frame
        cv2.imshow("Frame", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video capture and close windows
    cap.release()
    cv2.destroyAllWindows()

    # Calculate and print the mean of human and non-human detections
    mean_human_detections = np.mean(human_detections_per_frame)
    mean_non_human_detections = np.mean(non_human_detections_per_frame)
    print("Mean number of humans detected per frame:", mean_human_detections)
    print("Mean number of non-human objects detected per frame:", mean_non_human_detections)

    # Print the detection summary for the entire video
    print("Total humans detected in the video:", total_human_detections)
    print("Total non-human objects detected in the video:", total_non_human_detections)
    # Display the mean as the "correct" number of detections for demonstration
    print("Correct number of humans detected (based on mean):", round(mean_human_detections))
    print("Correct number of non-human objects detected (based on mean):", round(mean_non_human_detections))

    return all_pred_boxes

video_path = "C:\\Users\\TheEarthG\\Downloads\\human.mp4" 
weights_path = "C:\\Users\\TheEarthG\\Downloads\\yolov4.weights" 
config_path = "C:\\Users\\TheEarthG\\Downloads\\yolov4.cfg" 
class_names_path = "C:\\Users\\TheEarthG\\Downloads\\coco.names"

all_pred_boxes = detect_objects(video_path, weights_path, config_path, class_names_path)


In [44]:
import numpy as np
import pandas as pd

def calculate_iou(gt_box, det_box):
  """
  Calculates Intersection-over-Union (IoU) between two bounding boxes.

  Args:
      gt_box: Ground truth bounding box (list of [y_min, x_min, y_max, x_max] coordinates).
      det_box: Detected bounding box (list of [y_min, x_min, y_max, x_max] coordinates).

  Returns:
      IoU value (float) between 0 and 1. 
      IoU of 0.5 is considered as threshold to classify detection as true positive.
  """
  # Calculate area of overlap and area of union
  ymin_intersection = max(gt_box[0], det_box[0])
  xmin_intersection = max(gt_box[1], det_box[1])
  ymax_intersection = min(gt_box[2], det_box[2])
  xmax_intersection = min(gt_box[3], det_box[3])

  area_intersection = max(0, xmax_intersection - xmin_intersection) * max(0, ymax_intersection - ymin_intersection)
  area_gt = (gt_box[2] - gt_box[0]) * (gt_box[3] - gt_box[1])
  area_det = (det_box[2] - det_box[0]) * (det_box[3] - det_box[1])
  area_union = area_gt + area_det - area_intersection

  # Calculate IoU
  iou = area_intersection / area_union if area_union != 0 else 0 # Add a small value to avoid division by zero
  return iou

def evaluate_model(gt_bbox, pred_bbox, iou_threshold=0.5):
  
  """
  Evaluates a model's performance using Intersection over Union (IoU).
  
  Args:
    gt_bbox: A list of lists containing the ground truth bounding boxes for each image in the dataset. 
             Each bounding box should be a list of four integers representing [ymin, xmin, ymax, xmax].
    pred_bbox: A list of predicted bounding boxes in the same format as `gt_bbox`.
    
  Kwargs:
    iou_threshold: The minimum IoU required for a prediction to count as correct. Default is 0.5.

  Returns:
    - precision: Precision of the detection.
    - recall: Recall of the detection.
    - f1_score: F1 score of the detection.
  """
  #if the list of ground truth bounding box or prediction bounding box is empty then we can't calculate the precision, recall and f1.
  if len(gt_bbox) == 0 or len(pred_bbox) == 0:
        return 0, 0, 0
  
  gt_bbox = np.array(gt_bbox)
  pred_boxes = np.array(pred_bbox)

  true_positives = 0
  false_positives = 0
  false_negatives = 0

  for pred_box in pred_boxes:
    iou = calculate_iou(gt_bbox, pred_box)
    if np.max(iou) >= iou_threshold:
        true_positives += 1
    else:
        false_positives += 1

  false_negatives = len(gt_bbox) - true_positives

  precision = true_positives / (true_positives + false_positives)
  recall = true_positives / (true_positives + false_negatives)

  if precision == 0 or recall == 0:
    f1_score = 0
  else:
    f1_score = 2 * (precision * recall) / (precision + recall)

  return precision, recall, f1_score

def calculation_accuracy(detection_per_frame, ground_truth_per_frame):
   """
   param: detection_per_frame - list of integers representing number of human detections per frame.
   param: ground_truth_per_frame - list of integers representing number of actual humans in the frame.
   """
   if len(detection_per_frame) != len(ground_truth_per_frame):
      raise ValueError("Different number of detections and ground truths")
   
   correct_detections = sum(1 for detected, actual in zip(detection_per_frame, ground_truth_per_frame) if detected == actual)
   accuracy = correct_detections / len(ground_truth_per_frame) 
   return f'{accuracy*100:.2f}%'

def print_evaluation_table(detection_methods, results, results2):
    """
    Print a table with precision, recall, and F1 scores for multiple detection methods.

    Parameters:
    - detection_methods: List of names of detection methods.
    - results: Dictionary containing evaluation results for each method.
    """
    df = pd.DataFrame(results, index=['Precision', 'Recall', 'F1 Score'])
    df.columns.name = 'Detection Method'
    accuracy_df = pd.DataFrame(results2, index=['Human Detection Accuracy'])
    df = pd.concat([df, accuracy_df])
    print(df)

#Example usage. It supposes that you have list of bounding boxes coordinates both predicted and ground truth one.
gt_boxes = [[100, 100, 200, 200], [300, 300, 400, 400]]
pred_boxes_method1 = [[90, 90, 210, 210], [320, 320, 420, 420], [500, 500, 600, 600]]
pred_boxes_method2 = [[100, 100, 200, 200], [310, 310, 410, 410], [520, 520, 620, 620]]
iou_threshold = 0.5

# Example usage (assuming you have these lists from your video processing)
yolo_detections = [3, 2, 4, 3, 5]  # Example detection counts per frame from YOLO
hog_detections = [2, 2, 4, 3, 5]   # Example detection counts per frame from HOG
ground_truth = [3, 2, 4, 3, 5]     # Actual counts of humans per frame

results = {}
results['YOLOv4'] = evaluate_model(gt_boxes, pred_boxes_method1, iou_threshold)
results['HOG'] = evaluate_model(gt_boxes, pred_boxes_method2, iou_threshold)
# print_evaluation_table(['YOLOv4', 'HOG'], results)

results2={}
results2['YOLOv4'] = calculation_accuracy(yolo_detections, ground_truth)
results2['HOG'] = calculation_accuracy(hog_detections, ground_truth)
print_evaluation_table(['YOLOv4', 'HOG'], results, results2)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()